 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Cell-Formatting" data-toc-modified-id="Cell-Formatting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cell Formatting</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Data-Collection" data-toc-modified-id="Data-Collection-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Collection</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Variance-Filtering" data-toc-modified-id="Variance-Filtering-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Variance Filtering</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Weightening" data-toc-modified-id="Weightening-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Weightening</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Data-scalling" data-toc-modified-id="Data-scalling-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Data scalling</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Dendrogram-Formation" data-toc-modified-id="Dendrogram-Formation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dendrogram Formation</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Data-Display" data-toc-modified-id="Data-Display-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Display</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#generated-class-labels" data-toc-modified-id="generated-class-labels-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>generated class labels</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#plotting-dendrogram" data-toc-modified-id="plotting-dendrogram-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>plotting dendrogram</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#display-features-list-with-indexes" data-toc-modified-id="display-features-list-with-indexes-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>display features list with indexes</a></span></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#data-plot-per-cluster" data-toc-modified-id="data-plot-per-cluster-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>data plot per cluster</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/hiearchical_r.ipynb#Clustering-Metrics" data-toc-modified-id="Clustering-Metrics-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Clustering Metrics</a></span></li></ul></div>

# Cell Formatting

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Parameters

In [ ]:
DATASET_PATH = '../datasets/MargemEquatorial/full_scenario/dataste_analysis2018_01_30-14_48_00_filtered_dataset.csv'
VARIANCE_FILTER = 0
SCENARIO = ''

# Data Collection

In [ ]:
import pandas as pd
#from IPython.display import display

dataset = pd.read_csv(DATASET_PATH, delimiter=',')

target_labels = dataset['petrofacie']
petrofacies = list(set(target_labels))
dataset.index = [
    str(i) + '-' + str(petrofacie)
    for i, petrofacie in enumerate(dataset['petrofacie'])
]
targets = list(dataset['petrofacie'])

del dataset['petrofacie']
# del dataset['lamina:'] # Removing well ID

feature_names = list(dataset.columns.values)
# dataset.describe()

## Variance Filtering

In [ ]:
# print('from ' + str(dataset.shape[1]))
# dataset = dataset.loc[:,dataset.std()>VARIANCE_FILTER]
# print('to ' + str(dataset.shape[1]) + ' features')
# processed_features = dataset.shape[1]

In [ ]:
# dataset.describe()

## Weightening

In [ ]:
# dataset.loc[:, 'Main/single size mode(mm):'] *= 10

## Data scalling

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from copy import deepcopy

# scalled_dataset = deepcopy(dataset)
# dataset[:] = StandardScaler().fit_transform(X=dataset)


# Dendrogram Formation

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r

In [ ]:
N_CLUSTERS = 9

dendextend = robjects.packages.importr('dendextend')

distances = r.dist(dataset, method="manhattan")
dend = r['as.dendrogram'](r.hclust(distances, method="complete"))
dend = r.color_branches(dend, k=N_CLUSTERS)
dend = r.color_labels(dend, k=N_CLUSTERS)
dend = r.set(dend, 'branches_lwd', 3)
dend = r.set(dend, 'labels_cex', .8)

# Data Display

In [ ]:
from rpy2.robjects import r
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

## generated class labels
Indexes from the dataset in the order presented in the dendrogram

In [ ]:
leaves = list(r['order.dendrogram'](dend))
leaves = [leave - 1 for leave in leaves]

labels = list(r['cutree'](dend, k=N_CLUSTERS))
labels = [labels[leaf] for leaf in leaves]
set_labels = sorted(list(set(labels)), key=lambda x: labels.index(x))

## plotting dendrogram
Preprocessing to get a truthful representation of the classification and plot function in R

In [ ]:
branches_colors = list(r.get_leaves_branches_col(dend))
set_colors = sorted(
    list(set(branches_colors)), key=lambda x: branches_colors.index(x))

df = pd.DataFrame({
    'labels': set_labels,
    'colors': set_colors
}).sort_values('labels')
set_labels = df['labels']
set_colors = df['colors']
del df

%reload_ext rpy2.ipython
%R -w 1300  -i dend,set_labels,set_colors plot(dend); legend("topright", legend = set_labels, fill = set_colors);
display(dataset)

## display features list with indexes

In [ ]:
# pd.options.display.max_colwidth = 180
# display(pd.DataFrame(dataset.columns.values, columns=['FEATURE']))

## data plot per cluster
Boxplot features grouped according to the samples clustered

In [ ]:
# number_attributes = len(dataset.columns)
# clusters = []
# for label in set_labels:
#     cluster_indexes = [leaves[i] for i, x in enumerate(labels) if x == label]
#     cluster = np.matrix(dataset.iloc[cluster_indexes, :])
#     clusters.append(cluster)

#     plt.figure(figsize=(30, 7))
#     instances_labels = list(dataset.index[cluster_indexes])
#     ordered_petrofacies = [
#         instance_label.split('-')[1] for instance_label in instances_labels
#     ]
#     plt.suptitle("Cluster " + str(label) + '; ' + str(cluster.shape[0]) +
#                  ' instances: ' + ','.join(instances_labels))

#     plt.subplot(121)
#     if cluster.shape[
#             0] == 1:  # TEMPORARY WORKAROUND TO https://github.com/matplotlib/matplotlib/pull/8116
#         cluster = np.vstack((cluster[0, :], cluster[0, :]))
#     plt.boxplot(cluster, labeplt.figure(figsize=(30, 7))ls=range(0, len(dataset.columns)))
#     plt.xlabel('feature index')
#     plt.ylabel('percentage')
#     plt.yticks(np.arange(0, np.amax(cluster)+1, 2.0))

#     plt.subplot(122)
#     cmap = plt.get_cmap('tab10')
#     colors_mapping = dict(
#         zip(petrofacies, cmap(np.linspace(0, 1, len(petrofacies)))))
#     plot_colors = []
#     for petrofacie in ordered_petrofacies:
#         plot_colors.append(colors_mapping[petrofacie])

#     for petrofacie, color, instance in zip(ordered_petrofacies, plot_colors,
#                                            cluster):
#         instance_arr = np.squeeze(np.asarray(instance))
#         plt.plot(range(0, number_attributes),
#             instance_arr,
#             c=color,
#             linewidth=.5)
#         plt.scatter(
#             range(0, number_attributes),
#             instance_arr,
#             c=color,
#             label=petrofacie)

#     plt.xlabel('feature index')
#     plt.ylabel('percentage')
#     plt.xticks(np.arange(0, number_attributes, 1.0))
#     plt.yticks(np.arange(0, np.amax(cluster)+1, 2.0))
#     plt.grid()
#     plt.legend()

#     plt.show()

# Clustering Metrics

In [ ]:
from sklearn import metrics
from sklearn.metrics import silhouette_score, silhouette_samples

dataset = pd.read_csv(DATASET_PATH, delimiter=',')
dataset.index = [
    str(i) + '-' + str(petrofacie)
    for i, petrofacie in enumerate(dataset['petrofacie'])
]

ordered_labels = [-1]*len(targets)
for i, leaf in enumerate(leaves):
    ordered_labels[leaf] = labels[i]
    
ordered_labels =[label-1 for label in ordered_labels]

grouping_df_list = [pd.DataFrame() for label in ordered_labels]

for i, _ in enumerate(set(ordered_labels)):
    grouping_df_list[i] = grouping_df_list[i].append(dataset[[label == i for label in ordered_labels]])


In [ ]:
import papermill as pm

pm.record('hello', 'world')

pm.record('metric_adjusted_rand_score',  metrics.cluster.adjusted_rand_score(targets, ordered_labels))
pm.record('metric_adjusted_mutual_info_score', metrics.cluster.adjusted_mutual_info_score(targets, ordered_labels))
pm.record('metric_homogeneity_score', metrics.cluster.homogeneity_score(targets, ordered_labels))
pm.record('metric_completeness_score',metrics.cluster.completeness_score(targets, ordered_labels))
pm.record('metric_v_measure_score', metrics.cluster.v_measure_score(targets, ordered_labels))
pm.record('processed_features', processed_features)
pm.record('clusters_assignment', [group.to_dict() for group in grouping_df_list])